In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
from geopy.distance import distance
from datetime import datetime

cc_df = pd.read_csv('./fraud.csv')

pd.set_option('display.max_columns', 50)



# 요구사항
# 1. 사기거래 탐지에 필요한 컬럼만 선정함
# 2. 구매 금액, 시간, 거리에 대한 피쳐 엔지니어링을 진행함
# 3. 필요한 컬럼을 모두 수치형으로 인코딩함
# 4. 데이터 전처리 과정에 대한 자세한 설명과 함께 코드를 작성함


cc_df.head()



,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


In [115]:
# cc_df의 컬럼명과 자료형을 확인합니다.

cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

In [116]:
# cc_df의 컬럼별 통계량을 확인합니다.

cc_df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,4.911340e+05,491134.000000,491134.000000,491134.000000,491134.000000,4.911340e+05,4.911340e+05,491134.000000,491134.000000,491134.000000
mean,3.706013e+17,69.050120,50770.532384,37.931230,-90.495619,1.213922e+05,1.358730e+09,37.930272,-90.495411,0.002533
std,1.260229e+18,160.322867,26854.947965,5.341193,12.990732,3.725751e+05,1.819402e+07,5.372986,13.004100,0.050264
min,5.038744e+11,1.000000,1843.000000,24.655700,-122.345600,4.600000e+01,1.325376e+09,23.655789,-123.345106,0.000000
25%,2.131124e+14,8.960000,28405.000000,33.746700,-97.235100,1.228000e+03,1.343087e+09,33.781388,-96.984814,0.000000
50%,3.531130e+15,42.170000,49628.000000,38.507200,-87.591700,5.760000e+03,1.357257e+09,38.545124,-87.573441,0.000000
75%,4.653879e+15,80.330000,75048.000000,41.520500,-80.731000,5.083500e+04,1.374626e+09,41.624294,-80.685567,0.000000
max,4.956829e+18,25086.940000,99323.000000,48.887800,-69.965600,2.906700e+06,1.388534e+09,49.887523,-68.965624,1.000000


In [117]:
cc_df.shape #491134

(491134, 22)

In [118]:
#불필요한 컬럼 제거하기
cc_df.head(3)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0


In [119]:
# merchant 컬럼에 포함된 값이 몇 종류인지 확인합니다.

cc_df['merchant'].nunique()

693

In [120]:
# Q. job 컬럼에 포함된 값이 몇 종류인지 확인합니다.
# [[YOUR CODE]]
cc_df['job'].nunique()

110

In [121]:
# Q. cc_num 컬럼에 포함된 값이 몇 종류인지 확인합니다.

# [[YOUR CODE]]
cc_df['cc_num'].nunique()

124

In [122]:
# 불필요한 컬럼들을 제거합니다.

cc_df.drop(['merchant','first','last','street','city','state','zip','job','trans_num','unix_time'], axis = 1, inplace= True)

In [123]:
# cc_num 컬럼의 값을 기준으로 정렬합니다.

cc_df.sort_values('cc_num')

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
378075,2020-08-05 17:03:19,503874407318,shopping_pos,7.77,M,29.5894,-98.5201,1595797,1975-12-28,30.014539,-98.426767,0
230588,2019-12-20 22:21:36,503874407318,health_fitness,72.06,M,29.5894,-98.5201,1595797,1975-12-28,29.260487,-97.961746,0
421413,2020-10-10 12:39:32,503874407318,misc_pos,4.78,M,29.5894,-98.5201,1595797,1975-12-28,29.116928,-99.258936,0
468378,2020-12-13 15:55:44,503874407318,kids_pets,84.56,M,29.5894,-98.5201,1595797,1975-12-28,29.994537,-98.441297,0
345085,2020-06-22 23:52:06,503874407318,entertainment,24.33,M,29.5894,-98.5201,1595797,1975-12-28,29.277168,-98.239289,0
...,...,...,...,...,...,...,...,...,...,...,...,...
53631,2019-04-14 16:57:31,4956828990005111019,entertainment,27.41,M,40.6747,-74.2239,124967,1980-12-21,39.888460,-74.151440,0
485223,2020-12-27 14:14:40,4956828990005111019,home,28.52,M,40.6747,-74.2239,124967,1980-12-21,41.068207,-75.015316,0
264968,2020-02-15 13:32:48,4956828990005111019,shopping_pos,2.95,M,40.6747,-74.2239,124967,1980-12-21,41.435438,-73.839008,0
63475,2019-04-30 17:45:09,4956828990005111019,shopping_pos,7.28,M,40.6747,-74.2239,124967,1980-12-21,41.149072,-75.143152,0


In [130]:
# cc_num 컬럼의 값마다 데이터 개수를 계산합니다.

cc_df['cc_num'].value_counts()

cc_num
6538441737335434    4392
30270432095985      4392
4364010865167176    4386
4642255475285942    4386
6538891242532018    4386
                    ... 
36913587729122      3641
4681601008538160    3638
30551643947183      3638
4005676619255478    3638
3511378610369890    3628
Name: count, Length: 124, dtype: int64

In [131]:
# cc_num 컬럼의 값마다 amt의 평균, 표준편차를 계산하여 amt_info에 저장합니다.

amt_info = cc_df.groupby('cc_num')['amt'].agg(['mean','std']).reset_index()

In [132]:
amt_info.to_pickle('./amt_info.pkl')

In [133]:
# Q. cc_num 컬럼을 기준으로, cc_df와 amt_info 데이터를 합쳐서 cc_df에 저장합니다.
# (left merge를 수행합니다.)

# [[YOUR CODE]]
# amt_info
cc_df = cc_df.merge(amt_info, on='cc_num', how='left')


In [134]:
cc_df.shape #491134

(491134, 14)

In [135]:
# Q. 결제금액(amt)의 z-score를 계산하여 amt_z 컬럼에 저장합니다.
# 평균: mean, 표준편차: std

# [[YOUR CODE]]
# amt_info
cc_df['amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

In [136]:
# 사기 거래 데이터(is_fraud=1)를 확인합니다.

cc_df[cc_df['is_fraud'] == 1]

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,mean,std,amt_z
4794,2019-01-12 00:59:01,581686439828,gas_transport,11.73,M,41.5205,-80.0573,5507,1973-07-28,41.947427,-79.796264,1,58.578675,149.804992,-0.312731
4816,2019-01-12 03:48:07,581686439828,grocery_pos,328.68,M,41.5205,-80.0573,5507,1973-07-28,42.148618,-79.398595,1,58.578675,149.804992,1.803020
4979,2019-01-12 15:46:10,581686439828,food_dining,120.58,M,41.5205,-80.0573,5507,1973-07-28,42.470024,-80.126576,1,58.578675,149.804992,0.413880
5073,2019-01-12 19:53:59,581686439828,shopping_net,1081.35,M,41.5205,-80.0573,5507,1973-07-28,42.455406,-79.521640,1,58.578675,149.804992,6.827351
5124,2019-01-12 22:44:05,581686439828,shopping_net,776.70,M,41.5205,-80.0573,5507,1973-07-28,40.680209,-79.099101,1,58.578675,149.804992,4.793708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477832,2020-12-21 02:21:41,4716561796955522,grocery_pos,358.24,F,48.2777,-112.8456,743,1972-05-04,47.526202,-113.643313,1,52.537867,106.113023,2.880911
477847,2020-12-21 02:36:03,4716561796955522,shopping_net,859.12,F,48.2777,-112.8456,743,1972-05-04,48.272348,-112.328075,1,52.537867,106.113023,7.601161
479296,2020-12-21 22:38:38,4716561796955522,home,209.84,F,48.2777,-112.8456,743,1972-05-04,49.173669,-112.698767,1,52.537867,106.113023,1.482402
479305,2020-12-21 22:42:11,4716561796955522,food_dining,123.58,F,48.2777,-112.8456,743,1972-05-04,48.913048,-113.214921,1,52.537867,106.113023,0.669495


In [137]:
# z-score 계산이 완료되었으니 mean, std 컬럼을 제거합니다.

cc_df.drop(['mean','std'], axis = 1, inplace = True)

In [138]:
# cc_num, category 컬럼의 값마다 결제금액(amt)의 평균, 표준편차를 계산하여 cat_info에 저장합니다.

cat_info = cc_df.groupby(['cc_num','category'])['amt'].agg(['mean','std']).reset_index()

In [139]:
cat_info.to_pickle('./cat_info.pkl')

In [140]:
cc_df.head()
cc_df.shape

(491134, 13)

In [141]:
cat_info.head()
cat_info.shape

(1736, 4)

In [ ]:
# Q. cc_num 컬럼을 기준으로, cc_df와 cat_info 데이터를 합쳐서 cc_df에 저장합니다.
# (left merge를 수행합니다.)

# [[YOUR CODE]]
# cat_info
cc_df = cc_df.merge(cat_info, on=['cc_num', 'category'], how='left')

#cat_info가 .groupby(['cc_num', 'category'])집계에 의해 만들어졌으므로
#조인을 할 때도 기준으로 2개를 모두 넣어줘야한다.
#두 컬럼을 합쳐서 하나의 기준키로 사용하는 것.
#그렇지 않고 cc_num만 넣으면 중복으로 들어있는 cc_num에 모두 반응해서
#'행 폭발'이 일어난다. 이걸 놓쳐서 다시 찾으러 왔다.

In [ ]:
cc_df.shape #491134 여기다

(491134, 15)

In [144]:
# Q. 결제금액(amt)의 z-score를 계산하여 cat_amt_z 컬럼에 저장합니다.
# 평균: mean, 표준편차: std

# [[YOUR CODE]]
cc_df['cat_amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

In [145]:
# z-score 계산이 완료되었으니 mean, std 컬럼을 제거합니다.

# cc_df
cc_df.drop(['mean','std'], axis =1 , inplace = True)

In [146]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698


In [147]:
#결제 시간 관련 feature 분석
# Q. trans_date_trans_time 컬럼에서 시간 값을 추출하여 hour 컬럼에 저장합니다.
# (힌트: datetime 자료형을 이용합니다.)

# [[YOUR CODE]]
# cc_df.info()
cc_df['hour'] = pd.to_datetime(cc_df['trans_date_trans_time']).dt.hour

In [148]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0


In [149]:
# 결제시간을 morning, afternoon, night, evening으로 분류하기 위해 함수를 정의합니다.

def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'# 결제시간을 morning, afternoon, night, evening으로 분류하기 위해 함수를 정의합니다.

def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'

In [150]:
# Q. hour 컬럼에 hour_func 함수를 적용한 값을 hour_cat 컬럼에 저장합니다.

# [[YOUR CODE]]
cc_df['hour_cat'] = cc_df['hour'].apply(hour_func)

In [151]:
cc_df

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour,hour_cat
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0,evening
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0,evening
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0,evening
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0,evening
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0,evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,1.450980,23,evening
491130,2020-12-31 23:56:57,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,-0.128877,23,evening
491131,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,1.163822,23,evening
491132,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,0.440137,23,evening


In [152]:
# Q. 시간대별로 데이터 수를 계산하여 출력합니다.

# [[YOUR CODE]]
cc_df['hour_cat'].value_counts()

hour_cat
afternoon    176801
night        146697
evening       98662
morning       68974
Name: count, dtype: int64

In [153]:
# cc_num 컬럼의 값마다 amt 컬럼의 데이터 수를 계산하여 all_cnt 변수로 저장합니다.

all_cnt = cc_df.groupby('cc_num')['amt'].count().reset_index()

In [154]:
# cc_num, hour_cat 컬럼의 값마다 amt 컬럼의 데이터 수를 계산하여 hour_cnt 변수로 저장합니다.

hour_cnt = cc_df.groupby(['cc_num','hour_cat'])['amt'].count().reset_index()

In [155]:
all_cnt.head()

,cc_num,amt
0,503874407318,3655
1,567868110212,3644
2,571365235126,4374
3,581686439828,3653
4,630423337322,4362


In [156]:
hour_cnt.head()

,cc_num,hour_cat,amt
0,503874407318,afternoon,1280
1,503874407318,evening,737
2,503874407318,morning,558
3,503874407318,night,1080
4,567868110212,afternoon,1228


In [157]:
# Q. cc_num 컬럼을 기준으로 hour_cnt와 all_cnt를 합칩니다.
# (hour_cnt에서 left merge를 수행합니다.)

# [[YOUR CODE]]
hour_cnt = hour_cnt.merge(all_cnt, on='cc_num', how='left')

In [158]:
hour_cnt.head()

,cc_num,hour_cat,amt_x,amt_y
0,503874407318,afternoon,1280,3655
1,503874407318,evening,737,3655
2,503874407318,morning,558,3655
3,503874407318,night,1080,3655
4,567868110212,afternoon,1228,3644


In [159]:
# Q. amt_x와 amt_y 컬럼의 이름을 각각 hour_cnt, total_cnt로 변경합니다.

# [[YOUR CODE]]
hour_cnt = hour_cnt.rename(columns={'amt_x': 'hour_cnt', 'amt_y': 'total_cnt'})

In [160]:
hour_cnt.head()

,cc_num,hour_cat,hour_cnt,total_cnt
0,503874407318,afternoon,1280,3655
1,503874407318,evening,737,3655
2,503874407318,morning,558,3655
3,503874407318,night,1080,3655
4,567868110212,afternoon,1228,3644


In [161]:
# Q. 전체 거래 건수(total_cnt)에 대한 시간대별 거래 건수(hour_cnt)의 비율을 계산하여 
# hour_perc 컬럼에 저장합니다.

# [[YOUR CODE]]
hour_cnt['hour_perc'] = hour_cnt['hour_cnt'] / hour_cnt['total_cnt']

In [162]:
hour_cnt.head(10)

,cc_num,hour_cat,hour_cnt,total_cnt,hour_perc
0,503874407318,afternoon,1280,3655,0.350205
1,503874407318,evening,737,3655,0.201642
2,503874407318,morning,558,3655,0.152668
3,503874407318,night,1080,3655,0.295486
4,567868110212,afternoon,1228,3644,0.336992
5,567868110212,evening,820,3644,0.225027
6,567868110212,morning,529,3644,0.145170
7,567868110212,night,1067,3644,0.292810
8,571365235126,afternoon,1523,4374,0.348194
9,571365235126,evening,943,4374,0.215592


In [163]:
hour_cnt.tail(10)

,cc_num,hour_cat,hour_cnt,total_cnt,hour_perc
486,4861310130652566408,morning,529,3643,0.145210
487,4861310130652566408,night,1033,3643,0.283558
488,4906628655840914250,afternoon,1282,3655,0.350752
489,4906628655840914250,evening,800,3655,0.218878
490,4906628655840914250,morning,500,3655,0.136799
491,4906628655840914250,night,1073,3655,0.293570
492,4956828990005111019,afternoon,1277,3657,0.349193
493,4956828990005111019,evening,784,3657,0.214383
494,4956828990005111019,morning,558,3657,0.152584
495,4956828990005111019,night,1038,3657,0.283839


In [164]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour,hour_cat
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0,evening
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0,evening
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0,evening
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0,evening
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0,evening


In [165]:
hour_cnt = hour_cnt[['cc_num','hour_cat','hour_perc']]

In [166]:
hour_cnt.to_pickle('./hour_cnt.pkl')

In [167]:
# Q. cc_num, hour_cat 컬럼을 기준으로 cc_df와 hour_cnt 데이터를 합칩니다.
# (cc_df에서 left merge를 수행합니다.)

# [[YOUR CODE]]
cc_df = cc_df.merge(hour_cnt, on=['cc_num', 'hour_cat'], how='left')

In [168]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour,hour_cat,hour_perc
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0,evening,0.196470
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0,evening,0.214383
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0,evening,0.217252
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0,evening,0.213600
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0,evening,0.202882


In [169]:
# 시간 관련 feature 분석이 완료되었으므로 불필요해진 컬럼들을 제거합니다.

cc_df.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis =1 , inplace = True)

In [170]:
#거리 관련 feature 분석
# 이렇게 위도와 경도 값으로 두 지점 사이의 거리를 구할 수 있습니다.

distance((48.8878, -118.2105), (49.159047, -118.186462)).km

30.216618410410064

In [171]:
# 고객의 위치와 상점의 위치 사이의 거리를 계산하여 distance 컬럼에 저장합니다.
# (모든 데이터가 계산되므로 실행 시간이 오래 걸릴 수 있습니다.)

cc_df['distance'] = cc_df.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)

In [172]:
# 데이터 10000개만 계산해도 시간이 이렇게 걸립니다.

start_time =  datetime.now()
cc_df.head(10000).apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)
datetime.now() - start_time

datetime.timedelta(microseconds=889538)

In [173]:
cc_df.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244


In [ ]:
# Q. cc_num 컬럼의 값마다 distance의 평균, 표준편차를 계산하여 dist_info 변수에 저장합니다.

# [[YOUR CODE]]
dist_info = cc_df.groupby('cc_num')['distance'].agg(['mean', 'std']).reset_index()
# dist_info

# 여기서 뒤에 .reset_index()를 붙이지 않으면 cc_num 컬럼과 값들이 인덱스에 붙어있다.
# 컬럼쪽으로 빼주기 위해 리셋인덱스를 붙여준다.

In [175]:
dist_info.to_pickle('./dist_info.pkl')

In [180]:
# Q. cc_num 컬럼을 기준으로 cc_df와 dist_info 데이터를 합칩니다.
# (cc_df에서 left merge를 수행합니다.)

#주의사항 : 결과값에 대해서 shape문을 출력했을때 row가 200만개 이상으로 잡힌다면 잘못 merge되었습니다! 이 경우 LMS 초기화를 진행하고 코드를 수정하고 첫 코드블럭부터 실행해주세요!

# [[YOUR CODE]]
# dist_info
cc_df = cc_df.merge(dist_info, on='cc_num', how='left')


In [182]:
cc_df.head()
# cc_df.shape

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance,mean,std
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618,71.656621,28.090646
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605,74.811123,28.675031
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167,75.617531,28.784606
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745,74.706461,28.711493
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244,79.334924,29.620117


In [183]:
# Q. distance 값의 z-score를 계산하여 dist_z 컬럼에 저장합니다.

# [[YOUR CODE]]
cc_df['dist_z'] = (cc_df['distance'] - cc_df['mean']) / cc_df['std']

In [184]:
cc_df

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance,mean,std,dist_z
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618,71.656621,28.090646,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605,74.811123,28.675031,0.345370
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167,75.617531,28.784606,-0.272693
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745,74.706461,28.711493,-1.824486
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244,79.334924,29.620117,0.002138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,1.450980,0.209201,98.043837,77.251486,29.270739,0.710346
491130,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,-0.128877,0.207534,127.240424,77.491795,29.899826,1.663843
491131,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,1.163822,0.160339,100.023736,79.172599,30.051452,0.693848
491132,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,0.440137,0.218022,80.887812,72.656420,28.552151,0.288293


In [185]:
# 불필요해진 컬럼들을 제거합니다.

cc_df.drop(['lat','long','merch_lat','merch_long','mean','std'], axis = 1, inplace = True)

In [ ]:
#나이 feature 만들어보기
# Q. dob 컬럼에서 연도 값만 추출하여 dob 컬럼에 저장합니다.

# [[YOUR CODE]]
# cc_df.info()
cc_df['dob'] = pd.to_datetime(cc_df['dob']).dt.year

# 'dob'가 데이트타임인척하는 스트링이라 변환이 필요하다 
# 변환 후에도 .dt로 접근해주는 것이 포인트다
# 왜냐면 datetime 스칼라가 아니라 datetime 시리즈이기 때문에
# year같은 속성을 벡터화해서 쓰려고 .dt로 접근한다.

In [187]:
#범주형 데이터의 one-hot encoding
# 범주형 컬럼 'category'에 몇 종류의 값이 있는지 확인해봅시다.

cc_df['category'].nunique()

14

In [ ]:
# Q. cc_df의 범주형 데이터에 원-핫 인코딩을 적용합니다.
# (drop_first 옵션은 True로 설정합니다.)

# [[YOUR CODE]]
# cc_df.info()
# cc_df.head()

cc_df = pd.get_dummies(cc_df, drop_first=True)

#컬럼을 따로 지정할 수도 있지만
#그냥 넣어도 수치형이 아닌 것들을 알아서 인코딩한다
#표면적으로 수치형인데 개념저으로 범주형이라 인코딩하고 싶으면 수동으로 지정해야함

#데이트타임이 좀 특이한데 그냥 겟더미즈에 넣으면 반응 안하고 무시함
#근데 머신에 넣으면 오류남. 그래서 뭔가 변환을 해야하는데
#그냥 통째로 겟더미즈 수동 변환 컬럼에 지정해버리면 날짜 시간 값마다 더미 만들어서 차원폭발 일어남
#그래서 겟더미즈가 아니라 내가 데이트타임에서 원하는 값을 쪼개고 수치형으로 정리해놔야함


In [193]:
cc_df.head()

,cc_num,amt,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,630423337322,107.23,149,1978,0,0.321302,0.317631,0.196470,30.216618,-1.475224,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,4956828990005111019,44.71,124967,1980,0,-0.114637,-0.652060,0.214383,84.714605,0.345370,False,False,False,True,False,False,False,False,False,False,False,False,False,True
2,180048185037117,46.28,71485,1974,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,374930071163758,64.09,31515,1971,0,-0.001304,0.211780,0.213600,22.322745,-1.824486,False,False,False,True,False,False,False,False,False,False,False,False,False,True
4,2712209726293386,25.58,378909,1977,0,-0.393828,-0.575698,0.202882,79.398244,0.002138,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [194]:
# 이제 불필요해진 cc_num 컬럼을 제거합니다.

cc_df.drop('cc_num', axis = 1, inplace = True)